In [1]:
import pyvista

In [ ]:
pyvista.set_jupyter_backend("trame")

In [2]:
from __future__ import annotations

from cf_units import Unit
import geovista as gv
#import geovista.theme
from geovista.common import to_cartesian
import ipywidgets as widgets
import iris
import numpy as np
import pyvista as pv

In [3]:
# fname = "Fields_grid6_C1_T1_202310201800.txt"
# cubes = iris.load(fname, "CAESIUM-TRACER_AIR_CONCENTRATION")
# cube = cubes[0]

fname = "/data/users/nkristia/Reykjanes_May24/NAMEoutput_UKV_chemistryON_ForChris/Fields_grid1_C1_T197_202406020200.nc"
fname = "/data/users/itwl/geovista/adaq/nina/so2-air-concentration-timeseries.nc"

cubes = iris.load(fname)

In [4]:
cubes

[<iris 'Cube' of SULPHUR_DIOXIDE_AIR_CONCENTRATION / (g / m^3) (time: 216; altitude: 40; latitude: 300; longitude: 400)>]

In [5]:
cube = cubes[0]

In [6]:
cube

<iris 'Cube' of SULPHUR_DIOXIDE_AIR_CONCENTRATION / (g / m^3) (time: 216; altitude: 40; latitude: 300; longitude: 400)>

In [7]:
t = cube.coord("time")
z = cube.coord("altitude")
y = cube.coord("latitude")
x = cube.coord("longitude")

In [8]:
tmin, tmax, tidx = 1, t.shape[0] - 1, 1

In [9]:
print(f"{tmin=}, {tmax=}, {tidx=}")

tmin=1, tmax=215, tidx=1


In [10]:
z

<DimCoord: altitude / (m)  [ 125., 375., ..., 9625., 9875.]+bounds  shape(40,)>

In [11]:
y_cb = y.contiguous_bounds()
x_cb = x.contiguous_bounds()

z_cb = z.contiguous_bounds()
z_fix = np.arange(*z_cb.shape) * np.mean(np.diff(y_cb)) * 5

In [12]:
xx, yy, zz = np.meshgrid(x_cb, y_cb, z_fix, indexing="ij")
shape = xx.shape
print(f"{shape=}")

shape=(401, 301, 41)


In [13]:
xx.shape, yy.shape, zz.shape

((401, 301, 41), (401, 301, 41), (401, 301, 41))

In [14]:
clim = (np.min(cube[tidx].data), np.max(cube[tidx].data))

print(clim)

(0.0, 4.6409943e-07)


In [15]:
xyz = to_cartesian(xx, yy, zlevel=zz, zscale=0.005)

In [16]:
xyz.shape

(4948741, 3)

In [17]:
ggrid = pv.StructuredGrid(xyz[:, 0].reshape(shape), xyz[:, 1].reshape(shape), xyz[:, 2].reshape(shape))

In [18]:
ggrid

StructuredGrid,Information
N Cells,4800000
N Points,4948741
X Bounds,"2.953e-01, 8.435e-01"
Y Bounds,"-4.224e-01, 1.472e-01"
Z Bounds,"6.421e-01, 1.034e+00"
Dimensions,"401, 301, 41"
N Arrays,0


In [36]:
p = gv.GeoPlotter()

ggrid["data"] = np.ma.masked_less_equal(cube[tidx].data, 0).filled(np.nan).flatten()
# ggrid["opacity"] = 1 - (ggrid["data"] / clim[1])
p.add_mesh(ggrid.threshold(), scalars="data", name="plume", cmap="oxy", clim=clim, show_scalar_bar=False)  # , opacity="opacity", use_transparency=True)
p.add_base_layer(texture=gv.natural_earth_1())
p.show_axes()
p.show()

/var/tmp/ipykernel_87452/809920452.py:5: UserWarning: geovista found no coordinate reference system (CRS) attached to mesh.
  p.add_mesh(ggrid.threshold(), scalars="data", name="plume", cmap="oxy", clim=clim, show_scalar_bar=False)  # , opacity="opacity", use_transparency=True)


Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x7f23940504a0_3" src="http://vld173.cmpd1.metoffi…

In [37]:
pv.global_theme.allow_empty_mesh = True

In [40]:
cmin, cmax = clim
csteps = 1000000
cstep = (cmax - cmin) / csteps 
widgets.interact(callback_threshold, idx=widgets.FloatRangeSlider(value=clim, min=cmin, max=cmax, step=cstep, continuous_update=True, description="Threshold:", readout_format=".2e"))

interactive(children=(FloatRangeSlider(value=(0.0, 4.6409942910941027e-07), description='Threshold:', max=4.64…

<function __main__.callback_threshold(idx)>

In [41]:
widgets.interact(callback_time, tidx=widgets.IntSlider(value=tmin, min=tmin, max=tmax, step=1, continuous_update=False, description="Time:"))

interactive(children=(IntSlider(value=1, continuous_update=False, description='Time:', max=215, min=1), Output…

<function __main__.callback_time(tidx)>

In [38]:
global_threshold = 0

def callback_threshold(idx):
    global global_threshold
    global_threshold = idx
    
    mesh = ggrid.threshold(idx)
    mesh.set_active_scalars("data", preference="cell")
    p.add_mesh(mesh, name="plume", cmap="oxy", clim=clim, render=True, scalars="data", preference="cell", show_scalar_bar=False, show_edges=False)

In [39]:
def callback_time(tidx):
    ggrid["data"] = np.ma.masked_less_equal(cube[tidx].data, 0).filled(np.nan).flatten()
    mesh = ggrid.threshold(global_threshold)
    mesh.set_active_scalars("data", preference="cell")
    p.add_mesh(mesh, name="plume", cmap="oxy", clim=clim, render=True, scalars="data", preference="cell", show_scalar_bar=False, show_edges=False)

In [ ]:
result = ggrid.threshold().extract_geometry()

In [ ]:
result

In [ ]:
result.plot(cmap="matter")

In [ ]:
result.smooth(n_iter=350).plot(cmap="matter")

In [ ]:
result.smooth_taubin(n_iter=50, pass_band=0.05).plot(cmap="matter")